In [55]:
%matplotlib widget
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]

saveFigures is set to: True
Done loading packages


In [56]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

In [68]:
# Data is (only) in the file from the most recent tuesday. 
# Should be made smarter, but here hardcoded
# tuePath = 'SSI_data_2022-01-04'
tuePath = 'SSI_data_2022-01-11'

dirPath = path_data + tuePath + '\\'

df1 = pd.read_csv(dirPath+'gennembrudsinfektioner_table1.csv',delimiter=';')

df2_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_cases.csv',delimiter=';')
df2_H = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_indlagte.csv',delimiter=';')
df2_D = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_dode.csv',delimiter=';')
df2_R = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_repositive.csv',delimiter=';')
df2_Int = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_intensiv.csv',delimiter=';')

df3 = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_alle.csv',delimiter=';',decimal=",")
df3_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_cases.csv',delimiter=';',decimal=",")
df3_H = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_indlagte.csv',delimiter=';',decimal=",")
df3_D = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_dode.csv',delimiter=';',decimal=",")
df3_Int = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_intensiv.csv',delimiter=';',decimal=",")

In [74]:
df1

,Ugenummer,antal_personer_alle_Ingen vaccination,antal_personer_alle_Første vaccination,antal_personer_alle_Anden vaccination,antal_personer_alle_Fuld effekt efter primært forløb,antal_personer_alle_Fuld effekt efter revaccination,antal_personer_Ingen vaccination,antal_personer_Første vaccination,antal_personer_Anden vaccination,antal_personer_Fuld effekt efter primært forløb,...,antal_repositive_Ingen vaccination,antal_repositive_Første vaccination,antal_repositive_Anden vaccination,antal_repositive_Fuld effekt efter primært forløb,antal_repositive_Fuld effekt efter revaccination,antal_indlagte_Ingen vaccination,antal_indlagte_Første vaccination,antal_indlagte_Anden vaccination,antal_indlagte_Fuld effekt efter primært forløb,antal_indlagte_Fuld effekt efter revaccination
0,Uge 52-2021,1088909,248517,4538850,3151243,1347909,955932,219671,4230013,2902993,...,2454,233,4378,3657,668,356,42,738,457,275
1,Uge 51-2021,1120241,236376,4520221,3463159,1013327,995471,209289,4229920,3219251,...,1027,122,2397,2156,206,315,27,490,356,125
2,Uge 50-2021,1202421,177300,4498094,3636751,819218,1081507,154435,4226867,3403882,...,747,89,1777,1645,102,282,26,472,369,100
3,Uge 49-2021,1273414,130013,4474834,3817951,625942,1156534,108840,4219872,3592777,...,344,41,431,391,39,241,17,412,349,62
4,Uge 48-2021,1313210,109728,4456227,3944348,490284,1198994,89731,4212311,3724402,...,230,22,125,113,11,222,16,375,320,53
5,Uge 47-2021,1332367,102272,4444952,4026768,398226,1219539,83928,4209425,3811215,...,208,12,100,94,6,220,14,329,302,27
6,Uge 46-2021,1355547,89097,4435213,4093092,325196,1241318,73843,4205795,3879833,...,201,7,84,80,3,210,8,330,298,32
7,Uge 45-2021,1381461,72565,4426183,4165942,244589,1265138,60167,4200736,3953011,...,174,7,61,60,1,145,15,291,269,22
8,Uge 44-2021,1392712,68177,4419604,4242370,160769,1276661,56358,4197078,4028979,...,130,5,39,36,3,160,9,206,192,11
9,Uge 43-2021,1401365,67867,4412163,4278741,115497,1285531,56068,4192214,4066563,...,107,3,39,38,1,113,3,182,167,15


In [59]:
# df3_C
# df2_D
        
sNone = 'Ingen vaccination'        
sOne = 'Første vaccination'  
sTwo = 'Anden vaccination'
sFull = 'Fuld effekt efter primært forløb'   
sReva = 'Fuld effekt efter revaccination'


ageGroups = df2_C.Aldersgruppe.values
print(ageGroups)
# weekNames = df1.Ugenummer
weekNames = df1.Ugenummer.values[::-1]
weekNamesShort = [x[4:6] for x in weekNames]
wInt = [int(x[4:6]) for x in weekNames]
print(weekNames)

['0-5' '6-11' '12-15' '16-19' '20-29' '30-39' '40-49' '50-59' '60-64'
 '65-69' '70-79' '80+' '12+' 'Alle']
['Uge 01-2022' 'Uge 38-2021' 'Uge 39-2021' 'Uge 40-2021' 'Uge 41-2021'
 'Uge 42-2021' 'Uge 43-2021' 'Uge 44-2021' 'Uge 45-2021' 'Uge 46-2021'
 'Uge 47-2021' 'Uge 48-2021' 'Uge 49-2021' 'Uge 50-2021' 'Uge 51-2021'
 'Uge 52-2021']


In [60]:
# df1.iloc[:,40:50]

In [61]:
# Make function for gettings particular parts
def getTimeSeries(thisdf=df2_C,curStatus='Ingen vaccination',curAge='Alle',weekNames=weekNames):
    
    agedf = thisdf[thisdf.Aldersgruppe==curAge]
    allVals = []
    for curWeek in weekNames:
        toAdd = agedf[curWeek+'_'+curStatus].values[0]
        allVals.append(toAdd)
    allVals = np.array(allVals)

    return allVals

print(getTimeSeries(df2_H,sNone))
print(getTimeSeries(df2_H,sFull))

[346  32  38  53  73  90 113 160 145 210 220 222 241 282 315 356]
[392  54  42  54  75 136 167 192 269 298 302 320 349 369 356 457]


In [62]:
# Make a testplot of cases
allStatus = [sNone,sOne,sFull,sReva] # Omit "Two, but not full effect"
# allStatus = [sNone,sOne,sTwo,sFull,sReva] # Everything
cmap = plt.cm.get_cmap('Dark2',len(allStatus))
import matplotlib.colors
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","violet","blue"],len(allStatus))
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["purple","xkcd:mauve","green"],len(allStatus))
meanWidth = 5

# Set what to plot
curdf = df2_C
sYlabel = 'Tilfælde, antal'
# curdf = df2_D 
# sYlabel = 'Dødsfald, antal'
curdf = df3_C
sYlabel = 'Tilfælde, per 100.000'
# curdf = df3_D 
# sYlabel = 'Dødsfald per 100.000'
# curdf = df3_H 
# sYlabel = 'Indlæggelser per 100.000'
# curdf = df3_Int
# sYlabel = 'Intensiv, per 100.000'
# curAge = ageGroups[-4]
curAge = ageGroups[4]


fig,ax1 = plt.subplots()

ax1.set_title(curAge)
# for curStatus in allStatus:
for i in range(len(allStatus)):
    curStatus = allStatus[i]
    curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

    # ax1.plot(wInt,curTimeSeries,label=curStatus)
    
    # ax1.plot(wInt,curTimeSeries,'.:',linewidth=1,label=curStatus,color=cmap(i))
    # ax1.plot(rnTime(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=f'{meanWidth} ugers gennemsnit',color=cmap(i))

    ax1.plot(rnTime2(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

ax1.set_ylabel(sYlabel)


ax1.legend(loc='upper left')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,tight_layout=True)

# Set what to plot
curAge = ageGroups[-3]

curdf = df2_C
sYlabel = 'Antal'

ax = ax1 
ax.set_title('Smittetilfælde, '+curAge)
# for curStatus in allStatus:
for i in range(len(allStatus)):
    curStatus = allStatus[i]
    curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

    ax.plot(rnTime2(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
# sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

ax.set_ylabel(sYlabel)
ax.legend(loc='upper left')
ax.set_ylim(bottom=0)



curdf = df3_C
sYlabel = 'Per 100.000'
ax = ax2
for i in range(len(allStatus)):
    curStatus = allStatus[i]
    curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

    ax.plot(rnTime2(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
# sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

ax.set_ylabel(sYlabel)
ax.legend(loc='upper left')
ax.set_ylim(bottom=0)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 659.25)

In [64]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,tight_layout=True)

# Set what to plot
curAge = ageGroups[-3]

curdf = df2_H
sYlabel = 'Antal'

ax = ax1 
ax.set_title('Indlæggelser, '+curAge+f' årige, {meanWidth} ugers gennemsnit')
# for curStatus in allStatus:
for i in range(len(allStatus)):
    curStatus = allStatus[i]
    curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

    ax.plot(rnTime2(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
# sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

ax.set_ylabel(sYlabel)
ax.legend(loc='upper left')
ax.set_ylim(bottom=0)



curdf = df3_H
sYlabel = 'Per 100.000'
ax = ax2
for i in range(len(allStatus)):
    curStatus = allStatus[i]
    curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

    ax.plot(rnTime2(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
# sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

ax.set_ylabel(sYlabel)
ax.legend(loc='upper left')
ax.set_ylim(bottom=0)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 320.67600000000004)

In [65]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,tight_layout=True)

# Set what to plot
curAge = ageGroups[-3]

curdf = df2_D
sYlabel = 'Antal'

ax = ax1 
ax.set_title('Dødsfald, '+curAge+f' årige, {meanWidth} ugers gennemsnit')
# for curStatus in allStatus:
for i in range(len(allStatus)):
    curStatus = allStatus[i]
    curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

    ax.plot(rnTime2(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
# sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

ax.set_ylabel(sYlabel)
ax.legend(loc='upper left')
ax.set_ylim(bottom=0)



curdf = df3_D
sYlabel = 'Per 100.000'
ax = ax2
for i in range(len(allStatus)):
    curStatus = allStatus[i]
    curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

    ax.plot(rnTime2(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
# sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

ax.set_ylabel(sYlabel)
ax.legend(loc='upper left')
ax.set_ylim(bottom=0)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 125.25700000000002)

In [66]:

df1.columns[0:10]

Index(['Ugenummer', 'antal_personer_alle_Ingen vaccination',
       'antal_personer_alle_Første vaccination',
       'antal_personer_alle_Anden vaccination',
       'antal_personer_alle_Fuld effekt efter primært forløb',
       'antal_personer_alle_Fuld effekt efter revaccination',
       'antal_personer_Ingen vaccination', 'antal_personer_Første vaccination',
       'antal_personer_Anden vaccination',
       'antal_personer_Fuld effekt efter primært forløb'],
      dtype='object')

In [67]:
fig,ax1 = plt.subplots()


# allStatus = [sNone,sOne,sTwo,sFull,sReva] # Everything
# for curStatus in allStatus:
#     ax1.plot(weekNamesShort,df1['antal_personer_alle_'+curStatus].values[::-1],label=curStatus)
# ax1.plot(weekNamesShort,df1['antal_personer_alle_Ingen vaccination'].values[::-1])


curStatus = allStatus[0]
curVals = df1['antal_personer_alle_'+curStatus].values[::-1]
ax1.fill_between(weekNamesShort,curVals,label=curStatus)

curStatus = allStatus[1]
prevVals = curVals 
curVals = prevVals + df1['antal_personer_alle_'+curStatus].values[::-1]
ax1.fill_between(weekNamesShort,curVals,prevVals,label=curStatus)

curStatus = allStatus[2]
prevVals = curVals 
curVals = prevVals + df1['antal_personer_alle_'+curStatus].values[::-1]
ax1.fill_between(weekNamesShort,curVals,prevVals,label=curStatus)

curStatus = allStatus[3]
# prevVals = curVals 
curVals = prevVals + df1['antal_personer_alle_'+curStatus].values[::-1]
ax1.fill_between(weekNamesShort,curVals,prevVals,label=curStatus)
curStatus = allStatus[4]
prevVals = curVals 
curVals = prevVals + df1['antal_personer_alle_'+curStatus].values[::-1]
ax1.fill_between(weekNamesShort,curVals,prevVals,label=curStatus)

ax1.legend(loc='center left')
ax1.set_ylim(bottom=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: list index out of range

# Old tests below

In [ ]:
# thisdf = df2_C
# curStatus = 'Ingen vaccination'
# curAge = '0-5'

# weekNames = df1.Ugenummer
# agedf = thisdf[thisdf.Aldersgruppe==curAge]
# allVals = []
# for curWeek in weekNames:
#     toAdd = agedf[curWeek+'_'+curStatus].values[0]
#     allVals.append(toAdd)
# allVals = np.array(allVals)

# # for k in range(0,len(thisdf)):
# #     curRow = thisdf.loc[k]

In [ ]:
# # Flip dataframes to be like df1
# casedf = pd.DataFrame()
# casedf_once = pd.DataFrame()
# casedf_both = pd.DataFrame()
# casedf_full = pd.DataFrame()
# casedf_reva = pd.DataFrame()
# casedf_none = pd.DataFrame()

# weekNames = df1.Ugenummer
# casedf['Ugenummer'] = weekNames
# casedf_none['Ugenummer'] = weekNames
# casedf_once['Ugenummer'] = weekNames
# casedf_both['Ugenummer'] = weekNames
# casedf_full['Ugenummer'] = weekNames
# casedf_reva['Ugenummer'] = weekNames

# for k in range(0,len(df2_C)):
#     curRow = df2_C.loc[k]
#     curAge = df2_C.loc[k,'Aldersgruppe']
    
#     curNone = []
#     curOnce = []
#     curBoth = []
#     curFull = []
#     curReva = []
#     for curW in weekNames:
#         # print(curW)
        
#         thisNone = curRow[curW+'_Ingen vaccination']
#         curNone.append(thisNone)
        
#         thisOnce = curRow[curW+'_Første vaccination']
#         curOnce.append(thisOnce)
        
#         thisBoth = curRow[curW+'_Anden vaccination']
#         curBoth.append(thisBoth)
        
#         thisFull = curRow[curW+'_Fuld effekt efter primært forløb']
#         curFull.append(thisFull)
        
#         thisReva = curRow[curW+'_Fuld effekt efter revaccination']
#         curReva.append(thisReva)

    
#     # Add to dataframe
#     casedf[curAge+'_Ingen vaccination'] = curNone
#     casedf[curAge+'_Første vaccination'] = curOnce
#     casedf[curAge+'_Anden vaccination'] = curBoth 
#     casedf[curAge+'_Fuld effekt efter primært forløb'] = curFull
#     casedf[curAge+'_Fuld effekt efter revaccination'] = curReva
    
#     # Add to smaller dataframes as well
#     casedf_none[curAge] = curNone
#     casedf_once[curAge] = curOnce
#     casedf_both[curAge] = curBoth 
#     casedf_full[curAge] = curFull
#     casedf_reva[curAge] = curReva
    
# casedf